In [1]:
import numpy as np
# import jax.numpy as jnp

from THB.multilevel_spline_space import *
from THB.core import *
from THB.bspline_funcs import *

In [2]:
bs1 = BSpline(knotvector=np.array([0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1]), degree=2)
bs2 = BSpline(knotvector=np.array([0, 0, 0, 0.2, 0.4, 0.6, 0.8, 1, 1, 1]), degree=2)

tp = TensorProduct([bs1, bs2])
h_space = Space(tp, num_levels=3)
h_space.build_hierarchy_from_domain_sequence()

In [3]:
params = generate_parametric_coordinates((100, 100))
ctrl_pts = {lev: jnp.zeros((*(h_space.sh_fns[lev]), 2)) for lev in range(3)}
ctrl_pts[0] = grevilleAbscissae(h_space.sh_fns[0], h_space.degrees, h_space.knotvectors[0])

ac_cells_ac_supp = compute_active_cells_active_supp(h_space.cells, h_space.fns, h_space.degrees)
fn_coeffs = compute_refinement_operators(h_space.fns, h_space.Coeff, h_space.degrees)
params_cell_supp, num_supp = compute_active_span(params, h_space.knotvectors, h_space.cells, h_space.degrees, ac_cells_ac_supp)

# phi1 = THB_basis_fns_tp_parallel(params, params_cell_supp, fn_coeffs, h_space.sh_fns, h_space.knotvectors, h_space.degrees)
phi1, spans1, coeff1 = THB_basis_fns_tp_serial(params, params_cell_supp, fn_coeffs, h_space.sh_fns, h_space.knotvectors, h_space.degrees)

fn_coeffs, ctrl_pts_, Jm = pre_process_data(params, fn_coeffs, ctrl_pts, params_cell_supp, h_space.sh_fns, h_space.knotvectors, h_space.degrees)
repeat_ind = jnp.repeat(jnp.arange(params.shape[0]), jnp.array(num_supp))


100%|██████████| 10000/10000 [00:00<00:00, 931529.34it/s]
10000it [00:00, 20367.07it/s]


In [4]:
THB_jacobian(jnp.array(params), ctrl_pts_, jnp.array(fn_coeffs), repeat_ind, Jm, [jnp.array(kv) for kv in h_space.knotvectors[2]], h_space.degrees).block_until_ready()
THB_evaluate(jnp.array(params), ctrl_pts_, jnp.array(fn_coeffs), repeat_ind, Jm, [jnp.array(kv) for kv in h_space.knotvectors[2]], h_space.degrees).block_until_ready()

Array([[1.0000001e-05, 1.0000001e-05],
       [1.0009901e-02, 1.0000002e-05],
       [2.0009803e-02, 9.9999997e-06],
       ...,
       [9.7000027e-01, 9.9000001e-01],
       [9.8000014e-01, 9.9000007e-01],
       [9.9000007e-01, 9.9000007e-01]], dtype=float32)

In [5]:
# print(repeat_ind)
# phi2 = THB_basis_fns(jnp.array(params), jnp.array(fn_coeffs), repeat_ind, [jnp.array(kv) for kv in h_space.knotvectors[2]], h_space.degrees)
# jac = get_THB_jacobian(jnp.array(params), jnp.array(fn_coeffs), repeat_ind, [jnp.array(kv) for kv in h_space.knotvectors[2]], h_space.degrees)
pts = THB_evaluate(jnp.array(params), ctrl_pts_, jnp.array(fn_coeffs), repeat_ind, Jm, [jnp.array(kv) for kv in h_space.knotvectors[2]], h_space.degrees)
jac = THB_jacobian(jnp.array(params), ctrl_pts_, jnp.array(fn_coeffs), repeat_ind, Jm, [jnp.array(kv) for kv in h_space.knotvectors[2]], h_space.degrees)

In [6]:
jitted_jac = jit(THB_jacobian, static_argnums=6)


In [9]:
jitted_jac(jnp.array(params), ctrl_pts_, jnp.array(fn_coeffs), repeat_ind, Jm, [jnp.array(kv) for kv in h_space.knotvectors[2]], h_space.degrees)

Array([[[ 1.0000000e+00, -7.5783775e-11],
        [-5.8536682e-11,  1.0000000e+00]],

       [[ 1.0000000e+00, -2.1548281e-08],
        [ 8.8050470e-12,  9.9999988e-01]],

       [[ 1.0000001e+00, -4.0002277e-08],
        [ 1.8880633e-11,  1.0000001e+00]],

       ...,

       [[ 9.9999714e-01, -2.3841858e-06],
        [-1.4305115e-06,  9.9999523e-01]],

       [[ 9.9999714e-01, -1.4305115e-06],
        [-9.5367432e-07,  9.9999619e-01]],

       [[ 9.9999523e-01, -1.9073486e-06],
        [-3.8146973e-06,  9.9999571e-01]]], dtype=float32)

In [7]:
spans = [np.array(find_span_array_jax(params[:, dim], h_space.knotvectors[2][dim], h_space.degrees[dim])) for dim in range(h_space.ndim)]

fns1 = [[basisFun(spans[dim][i], params[i, dim], h_space.degrees[dim], h_space.knotvectors[2][dim]) for i in range(params.shape[0])] for dim in range(h_space.ndim)]

fns1 = np.einsum('ij, ik -> ijk', fns1[0], fns1[1])

In [8]:
phi2, fns2 = THB_basis_fns(params, params_cell_supp, num_supp, fn_coeffs, h_space.sh_fns, h_space.knotvectors, h_space.degrees, ctrl_pts)
# THB_evaluate_vmap(params, num_supp, fn_coeffs, ctrl_pts, Jm, h_space.knotvectors, h_space.degrees)
fns2 = np.array(fns2)

ValueError: Non-hashable static arguments are not supported. An error occurred during a call to 'THB_basis_fns' while trying to hash an object of type <class 'dict'>, {0: (7, 7), 1: (12, 12), 2: (22, 22)}. The error was:
TypeError: unhashable type: 'dict'


In [ ]:
fns2.shape

In [ ]:
phi2_sums = np.array([np.sum(sub_array) for sub_array in np.split(phi2, 10000)])
phi1_sums = np.array([np.sum(sub_array) for sub_array in np.split(phi1, 10000)])